This notebook contains code to compute the FFT of a list of data or a matrix, and the IFFT

The FFT (Fast Fourier Transform) is a method to compute the DFT (Discrete Fourier Transform), which is defined as:

$$
X_k = \sum_{n=0}^N-1 x_n \cdot e^{-i \tau k n / N}
$$

by splitting the input list (x) into its even and odd components (0-indexed), and then computing the DFT of each section (i.e recursively FFT until you get to single points, when the resultant DFT is just the value of the single point of data), then reconstruct using twiddle factors, defined as:

$$
e^{-i \tau n / N}
$$

The FFT has a big-O notation of O(n log n) rather than the DFT, which has a big-O notation of O(n^2), thus making it much better for computation than the DFT without any optimisations.

For the following implementation, a simple radix-2 FFT will be used, and this will require the input list/matrix to have dimensions that are a power of 2

The following code will add and use libraries required (feel free to comment out the adding packages section if the packages are already installed)

In [2]:
# Addition section
# using Pkg

# Pkg.add("Plots")
# Pkg.add("BenchmarkTools")

# Import section
using Plots
using BenchmarkTools

The following code will define functions that are used for convenience when doing an FFT (1-D or 2-D)

In [3]:
# Useful for both

# Checks if the integer supplied is a power of 2
function ispow2(x::Int64)
    return x > 0 && (x & (x - 1)) == 0
end

# Finds the next power of 2 above the supplied integer (returns the integer supplied if it is a power of 2)
function nextpow2(x::Int64)
    return ispow2(x) ? x : convert(Int, 2^ceil(log2(x)))
end

# 1-D FFT functions

# Pad a vector to a length of a power of 2
function pad_vector(x::Vector{T}, padder::T, l::Int64) where T <: Number
    if l <= length(x)
        return x
    end

    return vcat(x, [padder for i in length(x) + 1:l])
end

# 2-D FFT functions

# Makes displaying a matrix much more concise and neat as all entries truncated to 2.d.p
function format_matrix(A::Matrix{T}) where T <: Number
    if !(T <: Real)
        return map(x -> round(real(x), digits = 2) + im * round(imag(x), digits = 2), A)
    else
        return map(x -> round(x, digits = 2), A)
    end
end

# Pads matrix so both of its dimensions are a power of 2
function pad_matrix_for_fft(A::Matrix{Float64})
    m, n = size(A)
    m_padded, n_padded = nextpow2(m), nextpow2(n)
    A_padded = zeros(Float64, m_padded, n_padded)
    A_padded[1:m, 1:n] = A
    return A_padded
end

# Pads a matrix to custom dimensions, useful for FFT convolution 
function pad_matrix(A::Matrix{T}, m::Int64, n::Int64) where T <: Number
    A_padded = zeros(T, m, n)
    A_padded[1:size(A, 1), 1:size(A, 2)] = A
    return A_padded
end

# Returns a vector of the rows of the matrix
function rows(matrix::Matrix)
    m = size(matrix)[begin]

    return [matrix[i, begin:end] for i in 1:m]
end

# Returns a vector of the columns of the matrix
function columns(matrix::Matrix)
    n = size(matrix)[end]

    return [matrix[begin:end, i] for i in 1:n]
end


# Given a vector of rows or columns, this function will convert the vector to a matrix
# columns is used to specify whether the vector supplied is a vector of columns (true) or a vector of rows (false)
function convert_to_matrix(v::Vector{Vector{T}}, columns::Bool = false) where T <: Number
    if columns
        m, n = length(v[1]), length(v)
        matrix = zeros(T, m, n)
        for i in 1:n
            matrix[begin:end, i] = v[i]
        end
        return matrix
    else 
        m, n = length(v), length(v[1])
        matrix = zeros(T, m, n)
        for i in 1:m
            matrix[i, begin:end] = v[i]
        end
        return matrix
    end
end

convert_to_matrix (generic function with 2 methods)

The following functions are a 1-D FFT of a list of data, and then a 1-D IFFT of a list of data

Because of the following, when doing a 1-D FFT with a purely real input, one can optimise the calculation of the twiddle factors

$$
X_{N - k} 
\newline = \sum^{N-1}_{n=0} x_n \cdot e^{-i \tau n (N - k) / N}
\newline = \sum^{N-1}_{n=0} x_n \cdot 1 \cdot e^{-i \tau n k }
\newline = X_k^{\ast}
          
$$

In [17]:
function fft(x::Vector{T}) where T <: Number 
    N = length(x)

    if !ispow2(N)
        throw(ArgumentError("Ensure list is a length of a power of 2"))
    elseif N == 1
        return x
    end
    
    X_even = fft(x[1:2:end])
    X_odd = fft(x[2:2:end])
    factors::Vector{ComplexF64} = [] 
    
    if T <: Real
        factors_first = @. exp(-2pi * im * (0:N÷2) / N)
        factors_rest = reverse(@. conj(factors_first[2:end - 1]))
        factors = vcat(factors_first, factors_rest)
    else
        factors = @. exp(-2pi * im * (0:N-1) / N)
    end

    return vcat(X_even .+ factors[begin:N÷2] .* X_odd, X_even .+ factors[(1 + N÷2):end] .* X_odd)
end

function ifft(x::Vector{T}, normalise::Bool = true) where T <: Number
    N = length(x)

    if !ispow2(N)
        throw(ArugmentError("Ensure list is a length of a power of 2"))
    elseif N == 1
        return x
    end

    X_even = ifft(x[1:2:end])
    X_odd = ifft(x[2:2:end])
    factors = @. exp(2pi * im * (0:N-1) / N)

    return vcat(X_even .+ factors[begin:N÷2] .* X_odd, X_even .+ factors[1 + N÷2:end] .* X_odd)  ./ (normalise ? sqrt(N) : 1)
end

ifft (generic function with 2 methods)

The following functions are a 2D FFT followed by a 2D IFFT

A 2D FFT is an FFT along the columns, then applying an FFT along the rows of the resultant matrix.

A 2D IFFT is an IFFT along the columns, then applying an IFFT along the rows of the resultant matrix.

In [5]:
function fft2(A::Matrix{T}) where T <: Number
    m, n = size(A)

    if !(ispow2(m) && ispow2(n))
        throw(ArgumentError("Dimensions should be a power of 2"))
    end

    columns_fft_matrix = convert_to_matrix([fft(i) for i in columns(A)], true)
    return convert_to_matrix([fft(i) for i in rows(columns_fft_matrix)])
end

function ifft2(A::Matrix{T}) where T <: Number
    m, n = size(A)

    if !(ispow2(m) && ispow2(n))
        throw(ArgumentError("Dimensions should be a power of 2"))
    end

    columns_ifft_matrix = convert_to_matrix([ifft(i) for i in columns(A)], true)
    return convert_to_matrix([ifft(i) for i in rows(A)])
end

ifft2 (generic function with 1 method)

The FFT and IFFT are very useful for computing convolutions, because of the convolution theorem:

$$
\mathcal{F}\{x(t) \ast y(t)\} = \mathcal{F} \{x(t)\} \cdot \mathcal{F} \{y(t)\}
$$

thus meaning that:

$$
x(t) \ast y(t) = \mathcal{F}^{-1}\{\mathcal{F} \{x(t)\} \cdot \mathcal{F} \{y(t)\}\}
$$

and this applies to both lists and matrices.

The following code contains functions that execute convolutions of 1-D lists and 2-D matrices respectively.

In [6]:
function conv(list1::Vector{T}, list2::Vector{T}) where T <: Number
    length_required = nextpow2(length(list1) + length(list2) - 1)
    list1_padded = pad_vector(list1, 0.0, length_required)
    list2_padded = pad_vector(list2, 0.0, length_required)

    fft_1 = fft(list1_padded)
    fft_2 = fft(list2_padded)

    return (ifft(fft_1 .* fft_2) .* (list1[begin] == 0 || list2[begin] == 0 ? 2 : 1))[begin:length(list1) + length(list2) - 1] # When one of the lists start with 0 the result seems to be half of what is expected, so this compensates for that
end

function conv2(A::Matrix{T}, B::Matrix{T}) where T <: Number
    m, n = nextpow2(size(A)[1] + size(B)[1] - 1), nextpow2(size(A)[2] + size(B)[2] - 1)

    A_padded = pad_matrix(A, m, n)
    B_padded = pad_matrix(B, m, n)

    return ifft2(fft2(A_padded) .* fft2(B_padded))
end

conv2 (generic function with 1 method)

The following code will benchmark the FFT convolutions agains simply sliding a list/matrix over another, then multiplying terms and summing.

In [12]:
function normal_conv(A::Vector{T}, B::Vector{T}) where T <: Number
    result_length = length(A) + length(B) - 1
    result = zeros(T, result_length)

    for i in 1:length(A)
        for j in 1:length(B)
            result[i + j - 1] += A[i] * B[j]
        end
    end

    return result
end

function normal_conv2(A::Matrix{T}, B::Matrix{T}) where T <: Number
    result_rows = size(A, 1) + size(B, 1) - 1
    result_cols = size(A, 2) + size(B, 2) - 1
    result = zeros(T, result_rows, result_cols)

    for i in 1:size(A, 1)
        for j in 1:size(A, 2)
            for m in 1:size(B, 1)
                for n in 1:size(B, 2)
                    result[i + m - 1, j + n - 1] += A[i, j] * B[m, n]
                end
            end
        end
    end

    return result
end

normal_conv2 (generic function with 1 method)

In [37]:
vec1 = rand(Float64, 20000)
vec2 = rand(Float64, 30000)

@time begin
    print("Normal 1-D convolution: ")
    normal_conv(vec1, vec2)
end

@time begin
    print("FFT 1-D convolution: ")
    conv(vec1, vec2)
end

Normal 1-D convolution: 

  0.344796 seconds (8 allocations: 390.766 KiB)
FFT 1-D convolution: 

  0.226693 seconds (2.95 M allocations: 467.767 MiB, 12.71% gc time)


49999-element Vector{ComplexF64}:
 1.6531911515531907e-17 + 4.7818052425443124e-29im
 1.2387555319199782e-16 + 5.775317743385032e-29im
 1.6183922765982955e-16 + 1.1343317442953732e-28im
   2.17906064052662e-16 - 3.001878633615843e-29im
 3.7607467474242564e-16 + 1.4636531458001062e-28im
  4.382922336585561e-16 + 8.495983358102145e-30im
  5.530108896251086e-16 + 1.0366315116093642e-28im
  5.820432297977571e-16 + 1.7642616105680085e-28im
  5.523473046632519e-16 + 2.238288051864798e-28im
  5.943467442060901e-16 + 1.2881260673960432e-28im
                        ⋮
  5.540275685538607e-16 + 5.1982810651668875e-28im
  5.514724330217505e-16 + 3.8570557661037187e-28im
  6.017179246944783e-16 + 2.0067085325759312e-28im
  4.807063193173897e-16 + 6.233087991984692e-28im
  3.954551563858255e-16 + 8.241686919363089e-28im
 2.8541167391866234e-16 + 7.384768926378105e-28im
  2.211347327455393e-16 + 1.5708456237144254e-28im
 2.1454602497574828e-16 + 3.6538294992285546e-28im
 4.5616743912813924e-17 + 5.4

In [38]:
mat1 = rand(Float64, 1000, 100)
mat2 = rand(Float64, 5000, 500)

@time begin
    print("Normal 2-D convolution: ")
    normal_conv2(mat1, mat2)
end

@time begin
    print("FFT 2-D convolution: ")
    conv2(mat1, mat2)
end

The following code tests to see how much accuracy is lost after doing an FFT and then an IFFT of a list of data

In [ ]:
x = range(0, 10, length = 128)

y = @. sin(x)

y_after_processing = collect(map(x -> real(x), ifft(fft(y), false))) # remove any tiny imaginary parts to ensure each element is real

plot(x, [y, y_after_processing])